Hamza Mhadi

March 2021

In [39]:
import warnings 
warnings.filterwarnings("ignore")

In [40]:
import numpy as np 
import pandas as pd 
from pathlib import Path 
from collections import Counter

In [41]:
#Read the CSV into DataFrame

df = pd.read_csv("lending_data.csv")
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [42]:
#Dummy Encoding (Binary Encoded Data)

df_encoded = pd.get_dummies(df, columns=["homeowner"],drop_first=True)
df_encoded.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_own,homeowner_rent
0,10700,7.672,52800,0.431818,5,1,22800,low_risk,1,0
1,8400,6.692,43600,0.311927,3,0,13600,low_risk,1,0
2,9000,6.963,46100,0.349241,3,0,16100,low_risk,0,1
3,10700,7.664,52700,0.430740,5,1,22700,low_risk,1,0
4,10800,7.698,53000,0.433962,5,1,23000,low_risk,0,0


#Split the Data into Training and Testing

In [43]:
#Create our features

X = df_encoded.drop('loan_status',axis=1)


#Create our target
y = df_encoded['loan_status']


In [44]:
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_own,homeowner_rent
0,10700,7.672,52800,0.431818,5,1,22800,1,0
1,8400,6.692,43600,0.311927,3,0,13600,1,0
2,9000,6.963,46100,0.349241,3,0,16100,0,1
3,10700,7.664,52700,0.430740,5,1,22700,1,0
4,10800,7.698,53000,0.433962,5,1,23000,0,0


In [45]:
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [46]:
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [47]:
round(X.describe(),2)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_own,homeowner_rent
count,77536.00,77536.00,77536.00,77536.00,77536.00,77536.00,77536.00,77536.00,77536.0
mean,9805.56,7.29,49221.95,0.38,3.83,0.39,19221.95,0.40,0.1
std,2093.22,0.89,8371.64,0.08,1.90,0.58,8371.64,0.49,0.3
min,5000.00,5.25,30000.00,0.00,0.00,0.00,0.00,0.00,0.0
25%,8700.00,6.82,44800.00,0.33,3.00,0.00,14800.00,0.00,0.0
50%,9500.00,7.17,48100.00,0.38,4.00,0.00,18100.00,0.00,0.0
75%,10400.00,7.53,51400.00,0.42,4.00,1.00,21400.00,1.00,0.0
max,23800.00,13.24,105200.00,0.71,16.00,3.00,75200.00,1.00,1.0


In [48]:
#Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=101)

Data Pre-Processing

In [49]:
#Create the StandardScaler instance 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [50]:
# Fit the Standard Scaler with the training data 
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)


In [51]:
# Scale the training and testing data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Simple Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [53]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.947602318357393

In [54]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  561,    62],
       [   99, 18662]], dtype=int64)

In [55]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.90      0.99      0.87      0.95      0.89       623
   low_risk       1.00      0.99      0.90      1.00      0.95      0.90     18761

avg / total       0.99      0.99      0.90      0.99      0.95      0.90     19384



Oversampling

Naive Random Oversampling

In [56]:
#Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler

#View the count of target classes with Counter
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 56275, 'high_risk': 56275})

In [57]:
#Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression(solver='lbfgs', random_state=1)
model2.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [58]:
#Calculate the balanced accuracy score 


from sklearn.metrics import balanced_accuracy_score
y_pred = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946073798288739

In [59]:
#Display the confusion matrix 
from sklearn.metrics import confusion_matrix

y_pred = model2.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  620,     3],
       [  112, 18649]], dtype=int64)

In [60]:
 # Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



SMOTE Oversampling

In [61]:
#Resample the training data with SMOTE 

from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({'low_risk': 56275, 'high_risk': 56275})

In [62]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=1)
model3.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [63]:
#Calculate the balanced accuracy score 

from sklearn.metrics import balanced_accuracy_score
y_pred = model3.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9946073798288739

In [64]:
#Display the confusion matrix 
from sklearn.metrics import confusion_matrix

y_pred = model3.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  620,     3],
       [  112, 18649]], dtype=int64)

In [65]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



Undersampling

In [66]:
 # Use the SMOTEENN technique to perform combination sampling on the data
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 74082, 'low_risk': 74591})

In [69]:
 # Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [70]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.9946073798288739

In [71]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  620,     3],
       [  116, 18645]], dtype=int64)

In [72]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      0.99      0.99       623
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18761

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



Question (1) - Which model had the best balanced accuracy score? -All of them except for the simple Linear Regression 

Question (2) - Which model had the best recall score? - SMOTE

Question (3) - Which model had the best geometric mean score? - SMOTE